<a href="https://www.kaggle.com/code/hari141v/llm-zoomcamp-03-vector-search-homework?scriptVersionId=189070949" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### About
### llm-zoomcamp-03-vector-search-Homework

### Import necessary libraries and packages

In [2]:
pip install elasticsearch -qq 

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sentence_transformers==2.7.0 -qq

Note: you may need to restart the kernel to use updated packages.


In [4]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.14.0-linux-x86_64.tar.gz
!tar -xzf elasticsearch-8.14.0-linux-x86_64.tar.gz
!chown -R root:root elasticsearch-8.14.0
!adduser --disabled-password --gecos '' elasticsearch_user
!chown -R elasticsearch_user:elasticsearch_user elasticsearch-8.14.0

--2024-07-20 14:06:21--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.14.0-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 588482330 (561M) [application/x-gzip]
Saving to: 'elasticsearch-8.14.0-linux-x86_64.tar.gz'

elasticsearch-8.14. 100%[===================>] 561.22M  27.9MB/s    in 21s     

2024-07-20 14:06:42 (26.6 MB/s) - 'elasticsearch-8.14.0-linux-x86_64.tar.gz' saved [588482330/588482330]

Adding user `elasticsearch_user' ...
Adding new group `elasticsearch_user' (1000) ...
Adding new user `elasticsearch_user' (1001) with group `elasticsearch_user' ...
Creating home directory `/home/elasticsearch_user' ...
Copying files from `/etc/skel' ...


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import io
import os
import json
import requests
from tqdm.notebook import tqdm
import subprocess
import time
import json
import elasticsearch
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from IPython.display import clear_output
from IPython.display import display, Markdown

### Elasticsearch setup

In [6]:
# Run elasticsearch
command = "sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(20)

In [7]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

root         130      33  0 14:07 ?        00:00:00 /bin/sh -c sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1
root         131     130  0 14:07 ?        00:00:00 sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1
elastic+     132     131 36 14:07 ?        00:00:07 /kaggle/working/elasticsearch-8.14.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=./elasticsearch-8.14.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/kaggle/working/elasticsearch-8.14.0 -Des.path.conf=/kaggle/working/elasticsearch-8.14.0/config -Des.distribution.type=tar -cp /kaggle/working/elasticsearch-8.14.0/lib/*:/kaggle/working/elasticsearch-8.14.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher -E xpack.security.enabled=false -E thread_pool.se

In [8]:
es_client = Elasticsearch('http://localhost:9200/', request_timeout=60) 

In [16]:
!curl localhost:9200

{
  "name" : "1da592cc3716",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "_EJdhRzcTWOqEGFmK5JcOA",
  "version" : {
    "number" : "8.14.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "8d96bbe3bf5fed931f3119733895458eab75dca9",
    "build_date" : "2024-06-03T10:05:49.073003402Z",
    "build_snapshot" : false,
    "lucene_version" : "9.10.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


### Get sentence transformer model

In [17]:
model_name = 'multi-qa-distilbert-cos-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Sample Encode

In [43]:
v = model.encode('I just discovered the course. Can I still join it?',
                 #normalize_embeddings=True
                )

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

### Embedding dimension

In [44]:
v.shape

(768,)

### Q1. Getting the embeddings model
```

Create the embedding for this user question:

user_question = "I just discovered the course. Can I still join it?"

What's the first value of the resulting vector?

* -0.24
* -0.04
* 0.07
* 0.27

```

In [52]:
v[0]

0.07822265

### Q1 Answer :0.07

### Get document file from coure repo

In [18]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/documents-with-ids.json

--2024-07-20 14:08:09--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/documents-with-ids.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 699257 (683K) [text/plain]
Saving to: 'documents-with-ids.json'

documents-with-ids. 100%[===================>] 682.87K  --.-KB/s    in 0.03s   

2024-07-20 14:08:10 (26.5 MB/s) - 'documents-with-ids.json' saved [699257/699257]



### Load the document

In [19]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [23]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

### Filtering QnA only related to machine-learning-zoomcamp

In [20]:
documents=[d for d in documents if d['course']=='machine-learning-zoomcamp']
    

In [46]:
len(documents)

375

### Create an embedding for both question and answer fields

In [21]:
embeddings=[]
for doc in tqdm(documents):
    qt = f"{doc['question']} {doc['text']}"
    embeddings.append(model.encode(qt, show_progress_bar=False))
    doc['question_text_vector'] = model.encode(qt,show_progress_bar=False)

  0%|          | 0/375 [00:00<?, ?it/s]

### Document after adding vectors

In [27]:
for k in documents[0].keys():
    if k in ['question_text_vector']:
        print(k,'diemension:',len(documents[0][k]))
    else:
        print(k,':',documents[0][k])

text : Machine Learning Zoomcamp FAQ
The purpose of this document is to capture frequently asked technical questions.
We did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:
Data Engineering Zoomcamp FAQ
In the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo
work
section : General course-related questions
question : How do I sign up?
course : machine-learning-zoomcamp
id : 0227b872
question_text_vector diemension: 768


### Q2. Creating the embeddings
```
Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix X:

Create a list embeddings
Iterate over each document
qa_text = f'{question} {text}'
compute the embedding for qa_text, append to embeddings
At the end, let X = np.array(embeddings) (import numpy as np)
What's the shape of X? (X.shape). Include the parantheses
```

In [22]:
X=np.array(embeddings)

In [23]:
X.shape

(375, 768)

### Q2 Answer: (375, 768)

### Q3. Search

```
We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the cosine similarity, it's sufficient to multiply the matrix X by the vector v:

scores = X.dot(v)
What's the highest score in the results?

* 65.0
* 6.5
* 0.65
* 0.065
```

In [47]:
scores = X.dot(v)

In [48]:
np.max(scores)

0.6506574

### Q3 Answer: 0.65

In [24]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=5):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)

### Sample search results

In [49]:
custom_search_engine_result=search_engine.search(v, num_results=5)

In [50]:
[{key: value for key, value in item.items() if key != 'question_text_vector'} for item in custom_search_engine_result]


[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Get ground truth data(LLM generated question data) from course repo

In [27]:
df_ground_truth_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/ground-truth-data.csv'
df_ground_truth=pd.read_csv(df_ground_truth_url)
df_ground_truth=df_ground_truth[df_ground_truth['course']=='machine-learning-zoomcamp']
df_ground_truth.head()

,question,course,document
2123,Where can I sign up for the course?,machine-learning-zoomcamp,0227b872
2124,Can you provide a link to sign up?,machine-learning-zoomcamp,0227b872
2125,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0227b872
2126,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,0227b872
2127,How can I structure my questions and answers f...,machine-learning-zoomcamp,0227b872


### Convert each row to dictionary

In [28]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [51]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

### Hit-rate metric

In [29]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [31]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return  display(Markdown(f"## Hit_rate: {hit_rate(relevance_total)}"))

In [32]:
def custom_search_engine_question_vector(q):
    question = q['question']
    course = q['course']
    v_q = model.encode(question, show_progress_bar=False)
    return search_engine.search(v_q, num_results=5)

In [33]:
evaluate(ground_truth, custom_search_engine_question_vector)

  0%|          | 0/1830 [00:00<?, ?it/s]

## Hit_rate: 0.9398907103825137

### Q4. Hit-rate for our search engine
```
calculate the hitrate of VectorSearchEngine with num_results=5.

What did you get?

* 0.93
* 0.73
* 0.53
* 0.33
```

### Q4 Answer: Hit_rate = 0.93

### Create index for elasticsearch

In [34]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

### Add index to elasticsearch

In [35]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

### Sample search

In [36]:
query = 'I just discovered the course. Can I still join it?'

In [37]:
v_q = model.encode(query, show_progress_bar=False)

In [38]:
q={
    "field": "question_text_vector",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000, 
}
es_client.search(index=index_name, knn=q, 
                 source=["text", "section", "question", "course", "id"])["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'Y1p-0JABlu5EvVFi56ps',
  '_score': 0.825423,
  '_source': {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
   'section': 'General course-related questions',
   'question': 'The course has already started. Can I still join it?',
   'course': 'machine-learning-zoomcamp',
   'id': 'ee58a693'}},
 {'_index': 'course-questions',
  '_id': 'Zlp-0JABlu5EvVFi56q9',
  '_score': 0.735234,
  '_source': {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and

### Q5. Indexing with Elasticsearch
```
Now let's index these documents with elasticsearch

Create the index with the same settings as in the module (but change the dimensions)
Index the embeddings (note: you've already computed them)
After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?
```

### Q5 Answer: ee58a693

In [39]:
df_ground_truth[df_ground_truth['document']=='ee58a693']

,question,course,document
2193,"If I join the course late, can I still partici...",machine-learning-zoomcamp,ee58a693
2194,Will I be able to obtain a certificate if I jo...,machine-learning-zoomcamp,ee58a693
2195,How many course projects must I submit to be e...,machine-learning-zoomcamp,ee58a693
2196,What is the requirement for peer reviews to ge...,machine-learning-zoomcamp,ee58a693
2197,Can I submit homework if I join the course aft...,machine-learning-zoomcamp,ee58a693


### KNN vector search

In [40]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
       # "filter": {
      #      "term": {
     #           "course": course
       #     }
        #}
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

### Function to embed question

In [41]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question, show_progress_bar=False)

    return elastic_search_knn('question_text_vector', v_q, course)


### Evaluation result for question+text vector

In [42]:
evaluate(ground_truth, question_text_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

## Hit_rate: 0.9387978142076503

### Q6. Hit-rate for Elasticsearch
```
The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this is not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster.

Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

* 0.93
* 0.73
* 0.53
* 0.33
```

### Q6 Answer: Hit_rate = 0.93